In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import joblib

In [2]:
filename = "KRW-XRP-5m-full"

df_origin = pd.read_csv(f"../../data/{filename}.csv")

input_features = [
    "high_price",
    "low_price",
    "trade_price",
    "candle_acc_trade_volume",
    "rsi_14",
    "macd_histogram",

    # "stoch_k",
    # "stoch_d",
]

print(len(df_origin))

700000


In [3]:
def extract_time_features(df):
    # 필요 없는 컬럼 제거
    df.drop(columns=['market','candle_date_time_utc','candle_date_time_kst', 'opening_price', 'timestamp','candle_acc_trade_price'], inplace=True)
    
    # RSI, OBV 계산
    df['rsi_14'] = compute_rsi(df['trade_price'], window=14)
    # df['obv'] = compute_obv(df)
    
    # MACD 선 계산
    exp1 = df['trade_price'].ewm(span=12, adjust=False).mean()
    exp2 = df['trade_price'].ewm(span=26, adjust=False).mean()
    macd = exp1 - exp2
    
    # Signal 선 계산 (MACD 선의 9기간 EMA)
    macd_signal = macd.ewm(span=9, adjust=False).mean()

    # MACD 히스토그램 계산
    df['macd_histogram'] = macd - macd_signal
    
    # Stochastic Oscillator 추가 (%K, %D)
    # stoch_k, stoch_d = compute_stochastic(df, window=14, smooth_k=3, smooth_d=3)
    # df['stoch_k'] = stoch_k
    # df['stoch_d'] = stoch_d

    df.dropna(inplace=True)

    return df

def compute_obv(df):
    obv = [0]
    for i in range(1, len(df)):
        if df['trade_price'].iloc[i] > df['trade_price'].iloc[i-1]:
            obv.append(obv[-1] + df['candle_acc_trade_volume'].iloc[i])
        elif df['trade_price'].iloc[i] < df['trade_price'].iloc[i-1]:
            obv.append(obv[-1] - df['candle_acc_trade_volume'].iloc[i])
        else:
            obv.append(obv[-1])
    return obv

# 도움 함수 정의 (RSI, OBV, ATR, ROC)
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    rsi[avg_loss == 0] = 100  # Handle division by zero
    rsi[(avg_gain == 0) & (avg_loss == 0)] = 50  # Neutral if no gain/loss
    rsi = rsi.fillna(50)  # 추가: 남아있는 NaN을 50으로 채움
    return rsi

# preprocess.py 수정

# def create_tensors_from_dataframe(scaled_inputs, device):
#     import torch
#     # LSTM의 입력 형식: [batch_size, seq_length, input_size]
#     X = torch.tensor(scaled_inputs, dtype=torch.float32).to(device)  # [batch_size, seq_length, input_size]
#     return X


def compute_atr(df, window=14):
    """
    ATR (Average True Range)을 간단한 '단순 이동평균(rolling mean)'으로 계산합니다.
    필요에 따라 EMA나 Wilder’s 평균을 적용할 수도 있습니다.
    """
    # prev_close(전일 종가) 시프트
    prev_close = df['trade_price'].shift(1)

    # True Range 계산
    tr1 = df['high_price'] - df['low_price']
    tr2 = (df['high_price'] - prev_close).abs()
    tr3 = (df['low_price'] - prev_close).abs()
    tr = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)

    # ATR 계산 (rolling mean 버전)
    atr = tr.rolling(window=window, min_periods=window).mean()
    return atr

def compute_stochastic(df, window=14, smooth_k=3, smooth_d=3):
    """
    Stochastic Oscillator의 %K, %D를 계산합니다.
    - window: %K 계산 시 참조할 기간 길이
    - smooth_k: %K를 몇 번(며칠) 이동평균으로 스무딩할지 (기본 3)
    - smooth_d: %K를 다시 몇 번 이동평균해서 %D를 구할지 (기본 3)
    
    만약 더 간단하게 %K만 쓸 거라면 %D 계산 부분은 생략 가능합니다.
    """
    # rolling 최저/최고
    low_min = df['low_price'].rolling(window=window).min()
    high_max = df['high_price'].rolling(window=window).max()

    # %K (현재 종가가 (최저~최고) 어디쯤인지)
    # 100 * (종가 - 최저) / (최고 - 최저)
    # 분모가 0이 될 수 있으니 eps 추가
    eps = 1e-9
    stoch_k = 100 * ((df['trade_price'] - low_min) / (high_max - low_min + eps))

    # %K 스무딩
    if smooth_k > 1:
        stoch_k = stoch_k.rolling(window=smooth_k).mean()

    # %D = %K의 이동평균
    stoch_d = stoch_k.rolling(window=smooth_d).mean() if smooth_d > 0 else stoch_k
    
    return stoch_k, stoch_d

df = extract_time_features(df_origin)

In [4]:
# 스케일러 초기화
scaler_input = MinMaxScaler()
scaler_target = MinMaxScaler()

# 입력 피처 스케일링
scaled_inputs = scaler_input.fit_transform(df[input_features])

# 타겟 피처 스케일링 (여기서는 'trade_price'를 예측 대상으로 설정)
scaled_target = scaler_target.fit_transform(df[['trade_price']])

In [5]:
# 예시: 스케일링 전후 min/max 비교
print("Before scaling trade_price min:", df['trade_price'].min(), 
      ", max:", df['trade_price'].max())

scaled_trade_price = scaler_target.transform(df[['trade_price']])
print("After scaling trade_price min:", scaled_trade_price.min(), 
      ", max:", scaled_trade_price.max())

Before scaling trade_price min: 156.0 , max: 3996.0
After scaling trade_price min: 0.0 , max: 0.9999999999999999


In [6]:
# 스케일러 저장
joblib.dump(scaler_input, f"../../pickles/2025-01-14/{filename}-scaler_input.pkl")
joblib.dump(scaler_target, f"../../pickles/2025-01-14/{filename}-scaler_target.pkl")

['../../pickles/2025-01-14/KRW-XRP-5m-full-scaler_target.pkl']

In [7]:
# 시퀀스 길이 설정
SEQ_LENGTH = 60

# 데이터 준비
def create_sequences(inputs, targets, seq_length=SEQ_LENGTH):
    X = []
    y = []
    for i in range(len(inputs) - seq_length):
        X.append(inputs[i:i+seq_length])
        y.append(targets[i+seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_inputs, scaled_target, SEQ_LENGTH)

print(len(X), len(y))  # 시퀀스 수 확인
print(np.isnan(X).sum(), np.isnan(y).sum())  # NaN 개수 확인

699940 699940
0 0


In [8]:
print(len(X),len(y))

699940 699940


In [9]:
# X와 y를 npy 파일로 저장

length = len(X)
np.save(f"../../preprocessed/2025-01-14/{filename}-X.npy", X)
np.save(f"../../preprocessed/2025-01-14/{filename}-y.npy", y)